# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [3]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import os
from scipy.misc.pilutil import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
import keras

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size =10 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx =[x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    
                    temp = imresize(image,(y,z))
                    temp = temp/127.5-1 #Normalize data
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches

        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

#model A
A_model = Sequential()

A_model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=(30, 120, 120, 3), padding='same'))
A_model.add(BatchNormalization())
A_model.add(Activation('relu'))
A_model.add(MaxPooling3D(pool_size=(2,2,2)))

A_model.add(Conv3D(16,kernel_size=(3,3,3), padding='same'))
A_model.add(BatchNormalization())
A_model.add(Activation('relu'))
A_model.add(MaxPooling3D(pool_size=(2,2,2)))

A_model.add(Conv3D(32,kernel_size=(1,3,3), padding='same'))
A_model.add(BatchNormalization())
A_model.add(Activation('relu'))
A_model.add(MaxPooling3D(pool_size=(2,2,2)))

A_model.add(Conv3D(64, kernel_size=(1,3,3),padding='same'))
A_model.add(BatchNormalization())
A_model.add(Activation('relu'))
A_model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
A_model.add(Flatten())

A_model.add(Dense(1000, activation='relu'))
A_model.add(Dropout(0.5))
A_model.add(Dense(500, activation='relu'))
A_model.add(Dropout(0.5))

#softmax layer
A_model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
optimiser = optimizers.Adam(lr=0.001)
A_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (A_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [18]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
A_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 85s 1s/step - loss: 5.5810 - categorical_accuracy: 0.2915 - val_loss: 5.4490 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-5.62785-0.29110-5.44899-0.32000.h5
Epoch 2/10
67/67 [==============================] - 25s 377ms/step - loss: 3.5565 - categorical_accuracy: 0.2786 - val_loss: 8.7904 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-3.55653-0.27861-8.79042-0.24000.h5
Epoch 3/10
67/67 [==============================] - 26s 385ms/step - loss: 2.0159 - categorical_accuracy: 0.2587 - val_loss: 1.1653 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-2.01589-0.25871-1.16532-0.53000.h5
Epoch 4/10
67/67 [==============================] - 26s 391ms/step - loss: 1.6804 - 

In [14]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

#  model B
B_model = Sequential()
B_model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
B_model.add(Dropout(0.25))

B_model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
B_model.add(Dropout(0.25))

B_model.add(Flatten())
B_model.add(Dense(512, activation='relu'))
B_model.add(Dropout(0.5))
B_model.add(Dense(5, activation='softmax'))

B_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
B_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [15]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  Epoch 1/10train ; batch size = 10

67/67 [==============================] - 131s 2s/step - loss: 1.7473 - categorical_accuracy: 0.2173 - val_loss: 1.5376 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-1.75019-0.21267-1.53761-0.37000.h5
Epoch 2/10
67/67 [==============================] - 45s 676ms/step - loss: 1.5363 - categorical_accuracy: 0.3532 - val_loss: 1.5195 - val_categorical_accuracy: 0.3100

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-1.53631-0.35323-1.51947-0.31000.h5
Epoch 3/10
67/67 [==============================] - 45s 678ms/step - loss: 1.5958 - categorical_accuracy: 0.2289 - val_loss: 1.6121 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-1.59582-0.22886-1.61210-0.21000.h5
Epoch 4/10
67/67 [==============================] - 46s 679ms/step - loss: 1.6081 -

### Changing frame x,y,z values Experiment 1

In [17]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

# Define model B
B_model = Sequential()
B_model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
B_model.add(Dropout(0.25))

B_model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
B_model.add(Activation('relu'))
B_model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
B_model.add(Dropout(0.25))

B_model.add(Flatten())
B_model.add(Dense(512, activation='relu'))
B_model.add(Dropout(0.5))
B_model.add(Dense(5, activation='softmax'))

B_model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
B_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [18]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 71s 1s/step - loss: 1.6047 - categorical_accuracy: 0.2448 - val_loss: 1.4942 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-1.60537-0.24736-1.49424-0.35000.h5
Epoch 2/10
67/67 [==============================] - 22s 331ms/step - loss: 1.5630 - categorical_accuracy: 0.3333 - val_loss: 1.4888 - val_categorical_accuracy: 0.2600

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-1.56302-0.33333-1.48881-0.26000.h5
Epoch 3/10
67/67 [==============================] - 24s 361ms/step - loss: 1.4984 - categorical_accuracy: 0.3383 - val_loss: 1.4186 - val_categorical_accuracy: 0.3900

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-1.49845-0.33831-1.41862-0.39000.h5
Epoch 4/10
67/67 [==============================] - 23s 341ms/step - loss: 1.4027 - 

## Batch size to 20 Experiment 2

In [20]:
train_generator = generator(train_path, train_doc, 20)
val_generator = generator(val_path, val_doc, 20)

In [21]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val Source path =  ; batch size = 20train Epoch 1/10

; batch size = 20
67/67 [==============================] - 83s 1s/step - loss: 0.7928 - categorical_accuracy: 0.6807 - val_loss: 0.8617 - val_categorical_accuracy: 0.6400

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-0.81113-0.66929-0.86170-0.64000.h5
Epoch 2/10
67/67 [==============================] - 28s 416ms/step - loss: 0.7150 - categorical_accuracy: 0.7164 - val_loss: 0.8406 - val_categorical_accuracy: 0.6300

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-0.71497-0.71642-0.84064-0.63000.h5
Epoch 3/10
67/67 [==============================] - 29s 436ms/step - loss: 0.7445 - categorical_accuracy: 0.7512 - val_loss: 0.7872 - val_categorical_accuracy: 0.6300

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-0.74451-0.75124-0.78725-0.63000.h5
Epoch 4/10
67/67 [==============================] - 29s 434ms/step - loss: 0.8421 - 

### Batch size to 30 Experiment 3

In [22]:
train_generator = generator(train_path, train_doc, 30)
val_generator = generator(val_path, val_doc, 30)

In [23]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 30
Source path =  train ; batch size = 30
Epoch 1/10
67/67 [==============================] - 86s 1s/step - loss: 0.3662 - categorical_accuracy: 0.8487 - val_loss: 0.7938 - val_categorical_accuracy: 0.6625

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-0.35049-0.87673-0.79380-0.66250.h5
Epoch 2/10
67/67 [==============================] - 24s 359ms/step - loss: 0.3783 - categorical_accuracy: 0.8308 - val_loss: 1.0808 - val_categorical_accuracy: 0.5700

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-0.37834-0.83085-1.08077-0.57000.h5
Epoch 3/10
67/67 [==============================] - 24s 360ms/step - loss: 0.3591 - categorical_accuracy: 0.8607 - val_loss: 0.6769 - val_categorical_accuracy: 0.6900

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-0.35905-0.86070-0.67687-0.69000.h5
Epoch 4/10
67/67 [==============================] - 23s 339ms/step - loss: 0.3637 - 

### Batch size to 40 Experiment 4

In [24]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)

In [25]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 40
Source path =  train ; batch size = 40
Epoch 1/10
67/67 [==============================] - 181s 3s/step - loss: 0.2800 - categorical_accuracy: 0.8903 - val_loss: 0.8073 - val_categorical_accuracy: 0.6417

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-0.28228-0.89275-0.80734-0.64167.h5
Epoch 2/10
67/67 [==============================] - 124s 2s/step - loss: 0.2937 - categorical_accuracy: 0.8893 - val_loss: 0.9164 - val_categorical_accuracy: 0.5900

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-0.29353-0.88927-0.91644-0.59000.h5
Epoch 3/10
67/67 [==============================] - 119s 2s/step - loss: 0.2712 - categorical_accuracy: 0.8973 - val_loss: 0.8774 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-0.27124-0.89728-0.87743-0.62500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 

### Changing Optimizer to Adadelta Experiment 5

In [26]:
B_model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
B_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [27]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)


In [28]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 40
Source path =  train ; batch size = 40
Epoch 1/10
67/67 [==============================] - 184s 3s/step - loss: 0.3562 - categorical_accuracy: 0.8688 - val_loss: 0.9639 - val_categorical_accuracy: 0.6542

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-0.35760-0.86783-0.96385-0.65417.h5
Epoch 2/10
67/67 [==============================] - 120s 2s/step - loss: 0.2498 - categorical_accuracy: 0.9048 - val_loss: 1.0938 - val_categorical_accuracy: 0.6400

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-0.25220-0.90386-1.09376-0.64000.h5
Epoch 3/10
67/67 [==============================] - 118s 2s/step - loss: 0.1988 - categorical_accuracy: 0.9245 - val_loss: 1.1835 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-0.19885-0.92450-1.18347-0.67000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/10
67/67 [======

### Changing epoch to 20 Experiment 6

In [29]:
num_epochs = 20 # choose the number of epochs

In [30]:
B_model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
B_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [31]:
train_generator = generator(train_path, train_doc, 40)
val_generator = generator(val_path, val_doc, 40)

In [32]:
B_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 40
Source path =  train ; batch size = 40
Epoch 1/20
67/67 [==============================] - 178s 3s/step - loss: 0.1313 - categorical_accuracy: 0.9618 - val_loss: 1.4508 - val_categorical_accuracy: 0.6792

Epoch 00001: saving model to model_init_2020-05-2511_51_14.639699/model-00001-0.16567-0.95536-1.45076-0.67917.h5
Epoch 2/20
67/67 [==============================] - 120s 2s/step - loss: 0.0630 - categorical_accuracy: 0.9768 - val_loss: 1.6216 - val_categorical_accuracy: 0.6550

Epoch 00002: saving model to model_init_2020-05-2511_51_14.639699/model-00002-0.06272-0.97682-1.62164-0.65500.h5
Epoch 3/20
67/67 [==============================] - 121s 2s/step - loss: 0.0650 - categorical_accuracy: 0.9754 - val_loss: 1.6892 - val_categorical_accuracy: 0.6450

Epoch 00003: saving model to model_init_2020-05-2511_51_14.639699/model-00003-0.06502-0.97542-1.68924-0.64500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
67/67 [======

#### Till this point we achieved 99.39% accuracy in training data

### Experiment 7

In [14]:
# Changing back images per frame, height , width of image and batch size
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120 # image height
num_epochs = 20

In [15]:
C_Model = Sequential()
C_Model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,3), padding="same"))
C_Model.add(Activation('relu'))
C_Model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
C_Model.add(Activation('relu'))
C_Model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
C_Model.add(Dropout(0.25))

C_Model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
C_Model.add(Activation('relu'))
C_Model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
C_Model.add(Activation('relu'))
C_Model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
C_Model.add(Dropout(0.25))


C_Model.add(Flatten())
C_Model.add(Dense(512, activation='relu'))
C_Model.add(BatchNormalization())
C_Model.add(Dropout(0.5))
C_Model.add(Dense(5, activation='softmax'))
C_Model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
C_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
C_Model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = Source path =  train 10
Epoch 1/20
; batch size = 10
67/67 [==============================] - 243s 4s/step - loss: 1.7868 - categorical_accuracy: 0.3621 - val_loss: 4.1282 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-05-2514_11_14.784486/model-00001-1.79601-0.35897-4.12819-0.23000.h5
Epoch 2/20
67/67 [==============================] - 48s 710ms/step - loss: 1.6541 - categorical_accuracy: 0.3831 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2020-05-2514_11_14.784486/model-00002-1.65408-0.38308-13.53920-0.16000.h5
Epoch 3/20
67/67 [==============================] - 47s 705ms/step - loss: 1.7287 - categorical_accuracy: 0.3134 - val_loss: 3.7840 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-05-2514_11_14.784486/model-00003-1.72869-0.31343-3.78402-0.30000.h5
Epoch 4/20
67/67 [==============================] - 47s 702ms/step - loss: 1.8884

### Experiment 8

In [21]:
channel=3
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model D
D_model = Sequential()

D_model.add(Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
D_model.add(BatchNormalization())
D_model.add(Activation('relu'))
D_model.add(MaxPooling3D(pool_size=(2,2,2)))

D_model.add(Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
D_model.add(BatchNormalization())
D_model.add(Activation('relu'))

D_model.add(MaxPooling3D(pool_size=(2,2,2)))

D_model.add(Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
D_model.add(BatchNormalization())
D_model.add(Activation('relu'))

D_model.add(MaxPooling3D(pool_size=(2,2,2)))

D_model.add(Conv3D(nb_filters[3], kernel_size=(1,3,3), padding='same'))
D_model.add(BatchNormalization())
D_model.add(Activation('relu'))

D_model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
D_model.add(Flatten())

D_model.add(Dense(nb_dense[0], activation='relu'))
D_model.add(Dropout(0.5))

D_model.add(Dense(nb_dense[1], activation='relu'))
D_model.add(Dropout(0.5))

#softmax layer
D_model.add(Dense(nb_dense[2], activation='softmax'))
D_model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
D_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_8 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_14 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [22]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 10
D_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 78s 1s/step - loss: 4.3103 - categorical_accuracy: 0.2393 - val_loss: 1.6542 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2020-05-2514_11_14.784486/model-00001-4.32471-0.23831-1.65421-0.28000.h5
Epoch 2/10
67/67 [==============================] - 23s 347ms/step - loss: 2.2003 - categorical_accuracy: 0.2139 - val_loss: 1.4326 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to model_init_2020-05-2514_11_14.784486/model-00002-2.20033-0.21393-1.43256-0.35000.h5
Epoch 3/10
67/67 [==============================] - 26s 387ms/step - loss: 1.7891 - categorical_accuracy: 0.2786 - val_loss: 1.3818 - val_categorical_accuracy: 0.4800

Epoch 00003: saving model to model_init_2020-05-2514_11_14.784486/model-00003-1.78908-0.27861-1.38179-0.48000.h5
Epoch 4/10
67/67 [==============================] - 26s 386ms/step - loss: 1.6405 - 

### Experiment 9

In [23]:
channel=3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

input_shape=(x,y,z,channel)

# Define model
E_model = Sequential()

E_model.add(Conv3D(nb_filters[0], kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
E_model.add(BatchNormalization())
E_model.add(Activation('relu'))

E_model.add(MaxPooling3D(pool_size=(2,2,2)))

E_model.add(Conv3D(nb_filters[1],  kernel_size=(3,3,3), padding='same'))
E_model.add(BatchNormalization())
E_model.add(Activation('relu'))

E_model.add(MaxPooling3D(pool_size=(2,2,2)))

E_model.add(Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
E_model.add(BatchNormalization())
E_model.add(Activation('relu'))

E_model.add(MaxPooling3D(pool_size=(2,2,2)))

E_model.add(Conv3D(nb_filters[3],  kernel_size=(1,3,3),   padding='same'))
E_model.add(BatchNormalization())
E_model.add(Activation('relu'))

E_model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
E_model.add(Flatten())

E_model.add(Dense(nb_dense[0], activation='relu'))
E_model.add(Dropout(0.5))

E_model.add(Dense(nb_dense[1], activation='relu'))
E_model.add(Dropout(0.5))

#softmax layer
E_model.add(Dense(nb_dense[2], activation='softmax'))
E_model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
E_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_17 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_18 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [24]:
batch_size = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 10
E_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size =Source path =  train ; batch size =Epoch 1/10
 10 10

67/67 [==============================] - 78s 1s/step - loss: 6.0133 - categorical_accuracy: 0.2677 - val_loss: 2.2806 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2020-05-2514_11_14.784486/model-00001-6.04827-0.26697-2.28062-0.36000.h5
Epoch 2/10
67/67 [==============================] - 24s 355ms/step - loss: 2.2812 - categorical_accuracy: 0.2587 - val_loss: 2.1477 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-05-2514_11_14.784486/model-00002-2.28120-0.25871-2.14773-0.22000.h5
Epoch 3/10
67/67 [==============================] - 26s 387ms/step - loss: 1.7471 - categorical_accuracy: 0.3184 - val_loss: 1.2573 - val_categorical_accuracy: 0.4600

Epoch 00003: saving model to model_init_2020-05-2514_11_14.784486/model-00003-1.74706-0.31841-1.25726-0.46000.h5
Epoch 4/10
67/67 [==============================] - 26s 387ms/step - loss: 1.7567 - 

### Experiment 10

In [25]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
F_Model = Sequential()

F_Model.add(Conv3D(nb_filters[0],  kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
F_Model.add(BatchNormalization())
F_Model.add(Activation('relu'))

F_Model.add(MaxPooling3D(pool_size=(2,2,2)))

F_Model.add(Conv3D(nb_filters[1], kernel_size=(3,3,3),padding='same'))
F_Model.add(BatchNormalization())
F_Model.add(Activation('relu'))

F_Model.add(MaxPooling3D(pool_size=(2,2,2)))

F_Model.add(Conv3D(nb_filters[2],kernel_size=(1,3,3),padding='same'))
F_Model.add(BatchNormalization())
F_Model.add(Activation('relu'))

F_Model.add(MaxPooling3D(pool_size=(2,2,2)))

F_Model.add(Conv3D(nb_filters[3],  kernel_size=(1,3,3), padding='same'))
F_Model.add(Activation('relu'))
F_Model.add(Dropout(0.25))

F_Model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
F_Model.add(Flatten())

F_Model.add(Dense(nb_dense[0], activation='relu'))
F_Model.add(Dropout(0.5))

F_Model.add(Dense(nb_dense[1], activation='relu'))
F_Model.add(Dropout(0.5))

#softmax layer
F_Model.add(Dense(nb_dense[2], activation='softmax'))
F_Model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
F_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_21 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_22 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [26]:
batch_size = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 10
F_Model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 80s 1s/step - loss: 2.0605 - categorical_accuracy: 0.3224 - val_loss: 1.9769 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-05-2514_11_14.784486/model-00001-2.05598-0.32579-1.97693-0.23000.h5
Epoch 2/10
67/67 [==============================] - 23s 350ms/step - loss: 1.4413 - categorical_accuracy: 0.3682 - val_loss: 2.8547 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-05-2514_11_14.784486/model-00002-1.44126-0.36816-2.85467-0.25000.h5
Epoch 3/10
67/67 [==============================] - 26s 391ms/step - loss: 1.4501 - categorical_accuracy: 0.4328 - val_loss: 1.5969 - val_categorical_accuracy: 0.4100

Epoch 00003: saving model to model_init_2020-05-2514_11_14.784486/model-00003-1.45014-0.43284-1.59688-0.41000.h5
Epoch 4/10
67/67 [==============================] - 27s 402ms/step - loss: 1.2853 - 

### Final Model

In [27]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0],  kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], kernel_size=(3,3,3), padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], kernel_size=(1,3,3), padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3],  kernel_size=(1,3,3), padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [28]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_25 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_18 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_25 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_19 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_26 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [29]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [30]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  val ; batch size = 10
Source path =  train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 80s 1s/step - loss: 1.9147 - categorical_accuracy: 0.2582 - val_loss: 1.3843 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2020-05-2514_11_14.784486/model-00001-1.91822-0.26094-1.38427-0.28000.h5
Epoch 2/20
67/67 [==============================] - 24s 362ms/step - loss: 1.5515 - categorical_accuracy: 0.3184 - val_loss: 1.4405 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-05-2514_11_14.784486/model-00002-1.55146-0.31841-1.44051-0.30000.h5
Epoch 3/20
67/67 [==============================] - 27s 405ms/step - loss: 1.5521 - categorical_accuracy: 0.2687 - val_loss: 1.2982 - val_categorical_accuracy: 0.3700

Epoch 00003: saving model to model_init_2020-05-2514_11_14.784486/model-00003-1.55205-0.26866-1.29824-0.37000.h5
Epoch 4/20
67/67 [==============================] - 25s 378ms/step - loss: 1.6453 - 

In [31]:
batch_size = 40
num_epochs = 20

model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
67/67 [==============================] - 28s 413ms/step - loss: 0.9921 - categorical_accuracy: 0.5871 - val_loss: 0.8518 - val_categorical_accuracy: 0.7600

Epoch 00001: saving model to model_init_2020-05-2514_11_14.784486/model-00001-0.99212-0.58706-0.85176-0.76000.h5
Epoch 2/20
67/67 [==============================] - 26s 391ms/step - loss: 0.8884 - categorical_accuracy: 0.6468 - val_loss: 0.8064 - val_categorical_accuracy: 0.7100

Epoch 00002: saving model to model_init_2020-05-2514_11_14.784486/model-00002-0.88842-0.64677-0.80636-0.71000.h5
Epoch 3/20
67/67 [==============================] - 26s 381ms/step - loss: 0.8147 - categorical_accuracy: 0.6667 - val_loss: 0.8879 - val_categorical_accuracy: 0.6300

Epoch 00003: saving model to model_init_2020-05-2514_11_14.784486/model-00003-0.81475-0.66667-0.88793-0.63000.h5
Epoch 4/20
67/67 [==============================] - 26s 382ms/step - loss: 1.0194 - categorical_accuracy: 0.6020 - val_loss: 0.8244 - val_categorical_accurac